### Exercitiul 2

Pentru acest exercitui am implementat 2 variante ale metodei de normalizare.

Prima metoda foloseste libraria numpy si a 2-a foloseste scikit-learn.




In [ ]:
import numpy as np
def normalize_data(train_data, test_data, type=None):
  if type =='standard': 
    # standardizarea e obtinuta prin scaderea mediilor atributelor si impartirea la deviatiile standard.
    # Mediile si deviatiile standard sunt obtinute din datele de antrenare
    mean = np.mean(train_data,axis=0) # calculeaza mediile atributelor din datele de antrenare
    std = np.std(train_data,axis=0) # calculeaza deviatiile standard din datele de antrenare
    # Mean si std sunt 2 vectori de dimensiune egala cu numarul de atribute(features) din setul de date.
    # In cazul acestui laborator atributele sunt frecventele de cuvinte.
    train_data -= mean
    train_data/=std
    test_data -= mean
    test_data /=std
  elif type == 'l1':
    train_data = train_data / (np.expand_dims((np.linalg.norm(train_data,ord=1,axis=1)), axis=1) + 1e-6) # normalizarea L1.
    # 1e-6 este adunat pt cazurile in care norma e 0. In cazul acestui laborator, norma poate fi 0 daca un document nu are niciun cuvant
    # sau cuvintele pe care le contine nu sunt gasite in vocabular (acest al 2-lea caz fiind posibil pentru datele de test)
    # Iar np.expand_dims e folosit pentru a putea folosi broadcast(primul laborator pentru detalii) la impartire.
    # Mai exact, np.linalg.norm ne intoarce norma 1(pentru ca am setat ord =1) pentru fiecare exemplu din setul de date.
    # Deci output-ul este un vector de norme de dimensiune egala cu numarul de exemple din setul de date si pentru a folosi vectorul acesta la impartire e nevoie sa il
    # transformam in forma (nr_exemple, 1)
    test_data = test_data / (np.expand_dims((np.linalg.norm(test_data, ord=1,axis=1)), axis=1) + 1e-6)
  elif type == 'l2':
    norm_train = np.expand_dims((np.linalg.norm(train_data,ord=2,axis=1)), axis=1) # identic cu cazul l1, dar parametrul ord este setat la 2, pentru a calcula norma l2
    train_data = train_data / (norm_train + 1e-6)
    test_data = test_data / (np.expand_dims((np.linalg.norm(test_data, ord=2,axis=1)), axis=1) +1e-6)
  return train_data, test_data

In [ ]:
from sklearn.preprocessing import StandardScaler, Normalizer
def normalize_data_v2(train_data, test_data, type=None):
  if type=='standard':
    scaler = StandardScaler()
    scaler.fit(train_data)
    train_data = scaler.transform(train_data)
    test_data = scaler.transform(test_data)
  elif type=='l2':
    normalizer = Normalizer(norm='l2')
    train_data = normalizer.transform(train_data)
    test_data = normalizer.transform(test_data)
  elif type =='l1':
    normalizer = Normalizer(norm='l1')
    train_data = normalizer.transform(train_data)
    test_data = normalizer.transform(test_data)
  return train_data, test_data

### Exercitiul 3 si 4

In [ ]:
import copy
class BagOfWords:

  def __init__(self):
    self.vocab = {}
    self.words = []

  def build_vocabulary(self, data):
    # pentru fiecare cuvant din variabila "data" asociem un id unic,
    # care va reprezenta indexul cuvantului in vectorul de features
    id_word = 0
    for mesaj in data:
      for word in mesaj:
        if word not in self.vocab:
          self.vocab[word]=id_word
          id_word+=1
          self.words.append(word)
  
  def get_features(self, data):
    # transforma textele din var "data" in vectori de dimensiunea vocabularului
    # care contin frecventele cuvintelor. Deci feature_matrix este o matrice cu dimensiunile (nr de exemple, nr de cuvinte in vocabular)
    # si fiecare rand va contine frecventele cuvintelor pentru un anumit exemplu de propozitie.
    # variabila no_word este folosita pentru a afisa exemplele din setul data care nu contin niciun cuvant din vocabular
    feature_matrix = np.zeros((data.shape[0],len(self.vocab)))
    for i, example in enumerate(data):
      no_word = True
      for word in example:
        if word in self.vocab:
          feature_matrix[i,self.vocab[word]] +=1
          no_word = False
      if no_word:
        print(example)
    return feature_matrix

In [ ]:
train_data_original = np.load('./data/training_sentences.npy',allow_pickle=True)
train_labels = np.load('./data/training_labels.npy')
test_data_original = np.load('./data/test_sentences.npy',allow_pickle=True)
test_labels = np.load('./data/test_labels.npy')

In [ ]:
bow = BagOfWords() # cream o instanta a clasei BagOfWords
bow.build_vocabulary(train_data_original) # construim vocabularul folosind datele de antrenare. 
print(len(bow.vocab))

9522


### Exercitiul 5

Urmatoarele 2 celule reprezinta ex5.

Urmatoarea celula transforma datele din propozitii in vectori de frecventa. Iar exemplele pe care le printeaza sunt exemplele care nu contin niciun cuvant in vocabular-ul creat anterior.

In [ ]:
train_data_original = bow.get_features(train_data_original)
test_data_original = bow.get_features(test_data_original)

[]
[]
['Ultimately', 'tor', 'motive', 'tui', 'achieve', 'korli']
['22', '146tf150p']
['YO', 'YO', 'YO', 'BYATCH', 'WHASSUP']
['Ryder', 'unsoldnow', 'gibbs']
['Kkcongratulation']
['hanks', 'lotsly']
['645']


In [ ]:
train_data, test_data = normalize_data_v2(train_data_original,test_data_original,type='l2')

### Exercitiul 6

Pentru antrenarea modelului folosim functia fit.

Iar cu functia predict obtinem predictiile pentru setul de test.

In [ ]:
from sklearn import svm
svm_model = svm.SVC(C=1,kernel= "linear") # definirea modelului
svm_model.fit(train_data, train_labels) # antrenare

SVC(C=1, kernel='linear')

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

pred_labels = svm_model.predict(test_data) # ne intoarce predictiile pentru fiecare exemplu din test_data
print("Accuracy:", accuracy_score(test_labels,pred_labels),"F1-score:", f1_score(test_labels, pred_labels)) # acuratetea si scorul F1 se calculeaza din
                                                                                                            # predictii (pred_labels) si clasele reale(test_labels)

Accuracy: 0.9842391304347826 F1-score 0.9409368635437881


Mesajele spam au label-ul 1, iar cele non-spam au label-ul 0 (pentru noi 0 inseamna de fapt -1, asa categorisam exemplele negative). 

Functia de decizie a svm-ului avea forma wx+b = w1x1+w2x2+ ... + wnxn + b. Si spuneam ca predictia este 1(spam) daca wx+b este mai mare decat 0 si -1(non-spam) daca valoarea functiei este mai mica decat 0. Asta inseamna ca w-urile cu valorile ridicate corespund cuvintelor importante pentru predictia 1 (spam), iar w-urile cu valori mici(chiar negative) corespund cuvintelor care favorizeaza predictia -1(non-spam). Deci pentru a gasi cuvintele cele mai negative, ne uitam la coeficientii w si luam cuvintele care corespund unor w-uri mari (adica favorizeaza spam). Si invers pentru cuvintele cele mai pozitive, ne uitam la coeficientii w si luam cuvintele care corespund unor w-uri mici (adica favorizeaza non-spam).

Coeficientii(w) ii accesam cu atributul coef_[0], si putem folosi functia argsort din numpy pentru a lua indecsii (care corespund si cuvintelor) in ordinea crescatoare a coeficientilor. Apoi primele 10 si ultimele 10 cuvinte din vocabular (in ordinea inserarii lor) sunt cuvintele cerute de exercitiu.



#### Observatie:
 Acest rationament este valabil pentru ca folosim pentru svm kernel-ul 'linear', daca foloseam 'rbf' sau altceva era invalid deoarece functia de decizie nu mai avea aceeasi interpretabilitate.



In [ ]:
index_sort = np.argsort(svm_model.coef_[0])
bow.words = np.array(bow.words)
print("Most positive words:", bow.words[index_sort[:10]])
print("Most negative words:", bow.words[index_sort[-10:]])



Most positive words: ['&lt#&gt' 'me' 'i' 'Going' 'him' 'Ok' 'I' 'Ill' 'my' 'Im']
Most negative words: ['Text' 'To' 'mobile' 'CALL' 'FREE' 'txt' '&' 'Call' 'Txt' 'STOP']


In [ ]:
!unzip ./data_lab5.zip

Archive:  ./data_lab5.zip
  inflating: data/test_labels.npy    
  inflating: data/test_sentences.npy  
  inflating: data/training_labels.npy  
  inflating: data/training_sentences.npy  
